In [31]:
import torch
from torchvision import datasets
import torchvision.transforms as transform
from torch.utils.data import DataLoader, WeightedRandomSampler
import torch.optim as optim
import torch.nn as nn
import numpy as np
from ultralytics import YOLO
from torchvision.datasets import ImageFolder

In [32]:
class CustomAugmentedDataset(ImageFolder):

    # constructor
    def __init__(self, root, transforms_dict, default_transforms=None):
        super().__init__(root)
        self.transform_dict = transforms_dict
        self.default_transform = default_transforms

    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)

        class_name = self.classes[target]
        transform = self.transform_dict.get(class_name, self.default_transform)

        if transform:
            sample = transform(sample)
        return sample, target

In [33]:
default_transform = transform.Compose([
    transform.Resize(256),
    transform.CenterCrop(224),
    transform.ToTensor(),
    transform.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

strong_transform = transform.Compose([
    transform.Resize(256),
    transform.RandomResizedCrop(224),
    transform.RandomHorizontalFlip(),
    transform.RandomRotation(30),
    transform.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transform.ToTensor(),
    transform.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

small_classes = ['red-spot']
transform_dict = {cls: strong_transform for cls in small_classes}

# load dataset beserta transform-nya
train_dataset = CustomAugmentedDataset(root='/home/oz31/code/personal/python/tea/dataset/Train/', transforms_dict=transform_dict, default_transforms=default_transform)
val_dataset = CustomAugmentedDataset(root='/home/oz31/code/personal/python/tea/dataset/Valid/', transforms_dict=transform_dict, default_transforms=default_transform)

class_counts = np.bincount([label for _, label in train_dataset.samples])
class_weight = 1. / torch.tensor(class_counts, dtype=torch.float)

sample_weights = [class_weight[label] for _, label in train_dataset.samples]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_dataloader = DataLoader(train_dataset, batch_size=8, sampler=sampler, num_workers=6, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, num_workers=4, shuffle=False)

print(f"Class detected: {train_dataset.classes}")

print("Augmentation summary per class:")
for cls in train_dataset.classes:
    print(f"{cls.ljust(15)} → {'Strong' if cls in transform_dict else 'Default'}")

Class detected: ['algal_spot', 'brown-blight', 'gray-blight', 'healthy', 'helopeltis', 'leaf-rust', 'red-rust', 'red-spider-infested', 'red-spot', 'white-spot']
Augmentation summary per class:
algal_spot      → Default
brown-blight    → Default
gray-blight     → Default
healthy         → Default
helopeltis      → Default
leaf-rust       → Default
red-rust        → Default
red-spider-infested → Default
red-spot        → Strong
white-spot      → Default


In [34]:
# Load YOLO11 classification model with pre-trained weights
model = YOLO("yolo11n-cls.pt")  # Nano classification model, pre-trained on ImageNet
# Note: YOLO11 handles classifier adjustment internally during training, no manual layer modification needed

In [35]:
num_classes = len(train_dataset.classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('is cuda available?', torch.cuda.is_available())
model = model.to(device)

is cuda available? True


In [36]:
# Optimizer (for fine-tuning, applied to YOLO’s parameters)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)

In [37]:
num_epochs = 30

training_args = {
    'data': '../../dataset/Train/',  # Correct dataset path
    'epochs': num_epochs,
    'imgsz': 224,  # Match ResNet
    'batch': 8,
    'device': 0 if torch.cuda.is_available() else 'cpu',
    'workers': 0,  # Match your setup
    'project': './runs/train',
    'name': 'yolo11_cls_exp',
    'exist_ok': True,
    'pretrained': True,
    'optimizer': 'Adam',
    'lr0': 0.001,
    'patience': 50,
    # Augmentation settings (respecting red-spot's strong augmentation)
    'hsv_h': 0.015,  # Default hue
    'hsv_s': 0.7,    # Default saturation
    'hsv_v': 0.4,    # Default value
    'degrees': 10.0,  # Rotation
    'translate': 0.1, # Translation
    'scale': 0.5,    # Zoom
    'shear': 0.0,
    'flipud': 0.0,   # Vertical flip
    'fliplr': 0.5,   # Horizontal flip
    'mosaic': 0.0,   # Disable mosaic for classification
    'mixup': 0.0,    # Disable mixup
}

# Train the model
results = model.train(**training_args)

print(f"total batches: {len(train_dataloader)}")
best_val_loss = float('inf')

for epoch in range(num_epochs):
    print(f"starting epoch {epoch+1}/{num_epochs}")
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (images, labels) in enumerate(train_dataloader):
        if batch_idx % 100 == 0:
            print(f"processing batch {batch_idx+1}/{len(train_dataloader)}")
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / len(train_dataloader)
    train_accuracy = (correct / total) * 100
    print(f"epoch {epoch+1}/{num_epochs}, loss: {avg_loss:.4f}, accuracy: {train_accuracy:.2f}%")

    # validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    avg_val_loss = val_loss / len(val_dataloader)
    val_accuracy = (val_correct / val_total) * 100
    print(f"validation loss: {avg_val_loss:.4f}, accuracy: {val_accuracy:.2f}%")

Ultralytics 8.3.112 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1650 SUPER, 3875MiB)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=../../dataset/Train/, epochs=30, time=None, patience=50, batch=8, imgsz=224, save=True, save_period=-1, cache=False, device=0, workers=0, project=./runs/train, name=yolo11_cls_exp, exist_ok=True, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

train: Scanning /home/oz31/code/personal/python/training-skripsi/dataset/Train_split/train... 7937 images, 0 corrupt: 100%|██████████| 7937/7937 [00:01<00:00, 5635.29it/s]


train: New cache created: /home/oz31/code/personal/python/training-skripsi/dataset/Train_split/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 245.6±69.7 MB/s, size: 4.8 KB)


val: Scanning /home/oz31/code/personal/python/training-skripsi/dataset/Train_split/val... 1988 images, 0 corrupt: 100%|██████████| 1988/1988 [00:00<00:00, 6208.54it/s]

val: New cache created: /home/oz31/code/personal/python/training-skripsi/dataset/Train_split/val.cache
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 0 dataloader workers
Logging results to runs/train/yolo11_cls_exp
Starting training for 30 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

                   all       0.95          1

30 epochs completed in 0.814 hours.
Optimizer stripped from runs/train/yolo11_cls_exp/weights/last.pt, 3.2MB


Optimizer stripped from runs/train/yolo11_cls_exp/weights/best.pt, 3.2MB

Validating runs/train/yolo11_cls_exp/weights/best.pt...
Ultralytics 8.3.112 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1650 SUPER, 3875MiB)
YOLO11n-cls summary (fused): 47 layers, 1,538,834 parameters, 0 gradients, 3.2 GFLOPs
WARNING ⚠️ Dataset 'split=train' not found at /home/oz31/code/personal/python/training-skripsi/dataset/Train/train
Found 9518 images in subdirectories. Attempting to split...
Splitting /home/oz31/code/personal/python/training-skripsi/dataset/Train (10 classes, 9925 images) into 80% train, 20% val...
Split complete in /home/oz31/code/personal/python/training-skripsi/dataset/Train_split ✅
train: /home/oz31/code/personal/python/training-skripsi/dataset/Train_split/train... found 9506 images in 10 classes ✅ 
val: /home/oz31/code/personal/python/training-skripsi/dataset/Train_split/val... found 3557 images in 10 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 223/223 [00:17<00:00, 12.94it/s]


                   all       0.95          1
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/train/yolo11_cls_exp
processing batch 1/1241

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 2.640000104904175. Dividing input by 255.
0: 224x224 red-spider-infested 0.39, helopeltis 0.38, red-rust 0.19, red-spot 0.02, brown-blight 0.01, 2.1ms
1: 224x224 red-spider-infested 0.39, helopeltis 0.38, red-rust 0.19, red-spot 0.02, brown-blight 0.01, 2.1ms
2: 224x224 red-spider-infested 0.38, helopeltis 0.38, red-rust 0.19, red-spot 0.02, brown-blight 0.01, 2.1ms
3: 224x224 red-spider-infested 0.40, helopeltis 0.38, red-rust 0.18, red-spot 0.02, brown-blight 0.01, 2.1ms
4: 224x224 red-spider-infested 0.39, helopeltis 0.38, red-rust 0.19, red-spot 0.02, brown-blight 0.01, 2.1ms
5: 224x224 red-spider-infested 0.40, helopeltis 0.38, red-rust 0.18, red-spot 0.02, brown-blight 0.01, 2.1ms
6: 224x224 red-spider-infested 0

TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not list